In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# load data

In [3]:
import sys
sys.path.insert(0, "../research/fashion-mnist/")

In [4]:
from utils import mnist_reader
X_train, y_train = mnist_reader.load_mnist('../research/fashion-mnist/data/fashion', kind='train')
# X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [5]:
X_train.shape

(60000, 784)

# tsne

In [6]:
from openTSNE import TSNE, TSNEEmbedding, affinity, initialization
from openTSNE import initialization
from openTSNE.callbacks import ErrorLogger

In [7]:
%time aff = affinity.PerplexityBasedNN(X_train, n_jobs=8)

/home/antoine/anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/pynndescent/rp_trees.py", line 133:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  self.func_ir.loc))
/home/antoine/anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../..

CPU times: user 3min 15s, sys: 3.74 s, total: 3min 19s
Wall time: 1min 6s


In [8]:
P = aff.P

In [9]:
import scipy.sparse as sp

In [10]:
sp.diags(np.asarray(1 / P.sum(axis=1)).ravel()) @ P

<60000x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 8417446 stored elements in Compressed Sparse Row format>

In [14]:
%time init = initialization.pca(X_train, random_state=42)

CPU times: user 4.67 s, sys: 360 ms, total: 5.03 s
Wall time: 1.56 s


In [ ]:
embedding = TSNEEmbedding(
    init,
    aff,
    negative_gradient_method="fft",
    n_jobs=8,
)

In [ ]:
%%time 
embedding1 = embedding.optimize(n_iter=250, exaggeration=12, momentum=0.5)
embedding2 = embedding1.optimize(n_iter=750, exaggeration=1, momentum=0.8)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(embedding2[:,0], embedding2[:,1], c=y_train, rasterized=True)

# point-wise normalisation

In [11]:
class PointWiseNN(affinity.Affinities):
    def __init__(
        self,
        data,
        perplexity=30,
        method="approx",
        metric="euclidean",
        metric_params=None,
        symmetrize=True,
        n_jobs=1,
        random_state=None,
    ):
        self.n_samples = data.shape[0]
        self.perplexity = perplexity

        k_neighbors = min(self.n_samples - 1, int(3 * self.perplexity))
        self.knn_index, self.__neighbors, self.__distances = affinity.build_knn_index(
            data, method, k_neighbors, metric, metric_params, n_jobs, random_state
        )

        self.P = affinity.joint_probabilities_nn(
            self.__neighbors,
            self.__distances,
            [self.perplexity],
            symmetrize=symmetrize,
            n_jobs=n_jobs,
            normalization="point-wise",

        )

        self.n_jobs = n_jobs

In [12]:
%time aff_pointwise = PointWiseNN(X_train, n_jobs=8)

/home/antoine/anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/pynndescent/pynndescent_.py", line 38:
    @numba.njit(parallel=True, fastmath=True)
    def init_from_random(n_neighbors, data, query_points, heap, rng_state):
    ^

  self.func_ir.loc))
/home/antoine/anaconda3/envs/raviz_hybrid/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

F

CPU times: user 3min 15s, sys: 3.42 s, total: 3min 18s
Wall time: 1min 4s


In [15]:
embedding_ = TSNEEmbedding(
    init,
    aff_pointwise,
    negative_gradient_method="fft",
    n_jobs=8,
)

In [ ]:
%%time 
embedding1_ = embedding_.optimize(n_iter=250, exaggeration=12, momentum=0.5)
embedding2_ = embedding1_.optimize(n_iter=750, exaggeration=1, momentum=0.8)

MemoryError: 

Exception ignored in: 'openTSNE._tsne.estimate_negative_gradient_fft_2d'
MemoryError


MemoryError: 

Exception ignored in: 'openTSNE._tsne.estimate_negative_gradient_fft_2d'
Traceback (most recent call last):
  File "openTSNE/_tsne.pyx", line 575, in openTSNE._tsne.compute_kernel_tilde_2d
MemoryError


MemoryError: 

...

Exception ignored in: 'openTSNE._tsne.estimate_negative_gradient_fft_2d'
Traceback (most recent call last):
  File "openTSNE/_tsne.pyx", line 575, in openTSNE._tsne.compute_kernel_tilde_2d
MemoryError


MemoryError: 

Exception ignored in: 'openTSNE._tsne.estimate_negative_gradient_fft_2d'
Traceback (most recent call last):
  File "openTSNE/_tsne.pyx", line 575, in openTSNE._tsne.compute_kernel_tilde_2d
MemoryError


MemoryError: 

Exception ignored in: 'openTSNE._tsne.estimate_negative_gradient_fft_2d'
Traceback (most recent call last):
  File "openTSNE/_tsne.pyx", line 575, in openTSNE._tsne.compute_kernel_tilde_2d
MemoryError


In [ ]:

fig, ax = plt.subplots()
ax.scatter(embedding2[:,0], embedding2[:,1], c=y_train, rasterized=True)

# tsne - random sampling

In [ ]:
aff.P

In [ ]:
aff._PerplexityBasedNN__neighbors.shape

In [ ]:
aff._PerplexityBasedNN__distances.shape

In [ ]:
from openTSNE import _tsne

In [ ]:
__neighbors, __distances = aff._PerplexityBasedNN__neighbors, aff._PerplexityBasedNN__distances

In [ ]:
%time P = affinity.joint_probabilities_nn(__neighbors, __distances, [30])

In [ ]:
P

In [ ]:
a,b = _tsne.compute_gaussian_perplexity_tau(__distances, np.array([30], dtype=float))

In [ ]:
np.array(a).shape

In [ ]:
import scipy.sparse as sp

In [ ]:
n_samples = 60000
n_reference_samples = n_samples
k_neighbors = 90

a_ = np.array(a)

P_ = sp.csr_matrix(
    (
        a_.ravel(),
        __neighbors.ravel(),
        range(0, n_samples * k_neighbors + 1, k_neighbors),
    ),
    shape=(n_samples, n_reference_samples),
)


In [ ]:
P_

In [ ]:
60000*90

In [ ]:
(P_ + P_.T)

In [ ]:
P.indices, P.indptr, P.data

In [ ]:
P.data

In [ ]:
P.indptr.shape

In [ ]:
P.data.shape

In [ ]:
b = np.array(b)

In [ ]:
plt.hist(b)

In [ ]:
i, j = P.nonzero()

In [ ]:
from collections import Counter

In [ ]:
cnt_i = Counter(i)

In [ ]:
cnt_i

In [ ]:
cnt_j = Counter(j)

In [ ]:
cnt_j

In [ ]:
plt.hist(np.array([i for i in cnt_i.values()]))

In [ ]:
import stochasticsampling as ss

In [ ]:
%time affinity = ss.StochasticNN(X_train)

In [ ]:
%time init = initialization.pca(X_train, random_state=42)

In [ ]:
res = []


embedding = ss.TSNEEmbedding(
    init,
    affinity,
#     optimizer=openTSNE.tsne.gradient_descent(),
    optimizer=ss.gradient_descent(),
    callbacks=lambda it, err, emb: res.append((it, err)),
    callbacks_every_iters=1,
    n_jobs=8,
)

In [ ]:
embedding.optimize_?